## Import

In [ ]:
import os
import time
from datetime import datetime
import pandas as pd
import requests
import json
import re

from bs4 import BeautifulSoup
from user_agent import generate_navigator

import aiohttp
import asyncio

# jp.ID, jp.PW
from accounts import jp

## code start

- login

In [ ]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
HEADERS = {'User-Agent' : user_agent}

############################
# 보낼 데이터
############################
payload = {
    'user': {
        'email': jp.ID,
        'password': jp.PW,
        'remember_me': False  
    }
}

############################
# payload to json
############################
payload = json.dumps(payload)
payload

############################
# 헤더 생성
############################
while True:
    headers_ori = generate_navigator(device_type="desktop", os=('mac', 'linux', 'win'))
    if None not in headers_ori.values():
        break

############################
# 로그인용 헤더로 복사
############################
headers_login = dict(headers_ori)

############################
# 로그인 헤더에 값 추가
############################
headers_login['Referer'] = 'https://www.jobplanet.co.kr/users/sign_in?_nav=gb'
headers_login['Origin'] = 'https://www.jobplanet.co.kr'
headers_login['Content-Type'] = 'application/json'
headers_login['Accept'] = '*/*'
headers_login['Accept-Encoding'] = 'gzip, deflate, br'
headers_login['Accept-Language'] = 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7'
# headers_login['Content-Length'] = '100'

login_url = 'https://www.jobplanet.co.kr/users/sign_in'

############################
# 세션 생성
############################
session = requests.Session()

############################
# 로그인 쿠키와 로그인 헤더를 이용해 로그싱 포스트 요청
############################
res = session.post(login_url, data=payload, headers=headers_login)
html = res.text
print(html)
# html 값 확인 : 로그인 성공시 json 형태로 리턴됨
# {"success":true,"redirect_url":"/","di_resu":4445780}

- 회사 검색 후 첫번째 회사로 가져오기

In [ ]:
############################
# 회사 검색창 들어간 후 첫번째꺼 가져오는 함수
############################
def get_comp_review(url):
    #requests로 url 접근 요청
    r = requests.get(url, headers=HEADERS)

    #r변수에 저장된 content를 파싱해서 soup객체에  content 저장
    soup = BeautifulSoup(r.content, 'html.parser')

    #req에 저장한 html을 파싱해서 soup 이라는 변수에 저장
    #is_company_card 클래스인 div 태그에서 a 붙은 태그(a href 찾음)
    tag_text = soup.find('div', class_="is_company_card").find_all('a')

    # 찾은 a태그 애들중에 첫번쨰 a태그(첫번째 기업) str로 변환 후에 tag
    tag_text=str(tag_text[0])

    # '/'로 분할 후 3번째 문자열(찾은 태그에서 뽑고자 하는 기업 번호)
    comp_uid = tag_text.split('/')[2]

    # 찾고자 하는 기업번호가 추가된 최종 url
    new_url= f'https://www.jobplanet.co.kr/companies/{comp_uid}/reviews'

    #기업 번호 반환과 tag_return 튜플로 반환 튜플 인덱싱으로 uid 크롤링때는 tag_return, new_url은 접속용
    return get_comp_review_crawl(new_url,comp_uid)

In [ ]:
link_to_get_review = {
    'www.jobplanet.co.kr': get_comp_review #jopplanet url 인지 확인후 get_comp_review함수 실행시켜줄 딕셔너리
}

In [ ]:
############################
#회사 정보 요청 함수
############################
def get_info(url):
    #도메인주소만 가져오는 코드 www.jobplanet.co.kr
    site_url = url.split('/')[2]

    # 전역변수 딕셔너리에서 있는 키값이랑 site_url이 동일하면 value 아니면 None
    get_content_func = link_to_get_review.get(site_url, None)
    if get_content_func:
        return get_comp_review(url)
    else:
        return False, None

In [ ]:
############################
# 회사 이름 입력받아 url로 만들어주는 함수
############################
def get_url(comp):
    url = f'https://www.jobplanet.co.kr/search?query={comp}' #여기에 회사 이름 추가해야됨
    return get_info(url) 

# 로그인 성공한 res객체에서 쿠키 받아오기
cookie = res.cookies.get_dict()

# 새로운 세션 생성
session = requests.Session()
# 쿠키 추가
session.cookies.update(cookie)
# 원본 헤더 쿠키
session.headers.update(headers_ori)

- 크롤러

In [ ]:
############################
# 리뷰 크롤링 해오는 함수
############################

def get_comp_review_crawl(new_url, comp_uid):
    print("review_crawl 실행됨")
    review_list = []
    page = 1

    while True:
        # 현재 페이지의 URL 생성 및 요청
        url = f'https://www.jobplanet.co.kr/companies/{comp_uid}/reviews/?page={page}'
        print(url)
        # #requests로 url 접근 요청
        r = requests.get(url, headers=HEADERS)

        # r변수에 저장된 content를 파싱해서 soup객체에  content 저장
        soup = BeautifulSoup(r.content, 'html.parser')
        #추출해야할 정보들 죄다 zip로 묶어줌 uid는 저위에 comp_uid

        # 회사 uid
        # uid는 get_comp_review 함수에서 인자로 받음

        no_review_message = soup.select_one("#viewReviewsList > div > div > div > article > p")
        if no_review_message and "등록된 기업리뷰가 없습니다." in no_review_message.get_text():
            print("기업리뷰없음.")
            break

        # 리뷰들을 돌면서 데이터 추출
        reviews = soup.select("section > div > div.ctbody_col2 > div")

        for review in reviews:
            review_data = {}


            # 리뷰 작성자 정보
            # 직무
            elements_reviewer_info = soup.find_all('span', class_="txt1")
            reviewer_texts = [element_reviewer_info.get_text() for element_reviewer_info in elements_reviewer_info]

            positions = []
            is_offices = []
            review_dates = []

            for i in range(0, len(reviewer_texts), 4):
                position_elements = reviewer_texts[i] if len(reviewer_texts) > i else None
                position = re.sub(r"\s", "", position_elements) if position_elements else None
                positions.append(positions)

                is_office_elements = reviewer_texts[i+1] if len(reviewer_texts) > i+1 else None
                is_office = re.sub(r"\s", "", is_office_elements) if is_office_elements else None
                is_offices.append(is_office)

                review_date_elements = reviewer_texts[i+3] if len(reviewer_texts) > i+3 else None
                review_date = re.sub(r"[^0-9]", "", review_date_elements) if review_date_elements else None
                review_dates.append(review_date)

            print(positions)
            print(is_offices)
            print(review_dates)

            # 별점
            elements_review_rate = soup.find_all('div', class_="star_score")

            review_rates = []
            
            for element in elements_review_rate:
                style_value = element.get('style')
                if style_value is not None:
                    match = re.search(r"\d+", style_value)
                    if match is not None:
                        rate = float(match.group()) / 20
                        review_rates.append(rate)

            print(review_rates)

            # 장점 단점
            # elements_reviewer_info = soup.find_all('span', class_="txt1")

            # review_conts = []
            # review_sentis = []
            # texts = [element_reviewer_info.get_text() for element_reviewer_info in elements_reviewer_info]
            # print(texts)
            # good_review_cont_elements = soup.select('section > div > div.ctbody_col2 > div > dl > dd:nth-child(2) > span')
            # if good_review_cont_elements:
            #     good_review_cont = clean_str(good_review_cont_elements[0].get_text().strip())
            
            # bad_review_cont_elements = soup.select('section > div > div.ctbody_col2 > div > dl > dd:nth-child(4) > span')
            # if bad_review_cont_elements:
            #     bad_review_cont = clean_str(good_review_cont_elements[0].get_text().strip())

            review_list.append(review_data)

            comp_review_dict={
                'comp_uid' : comp_uid,
                # 'review_cont': review_cont,
                # 'review_senti' : review_senti,
                'good_review_cont': good_review_cont,
                'bad_review_cont': bad_review_cont,
                'review_rate' : review_rate,
                'is_office' : is_office,
                'review_date': review_date,
                'position' : position,
                }

        page += 1

    #기업 정보 key, value로 담은 딕셔너리
    return get_comp_review_df(comp_review_dict) #기업정보들 튜플로 반환

- Dataframe

In [ ]:
def get_comp_review_df(comp_review_dict):
    print("df 실행됨")
    # save_path = f"/Users/gyeonmunju/Desktop/PlayData/프로젝트_정리파일/request_change"
    # os.makedirs(save_path, exist_ok=True)
    df = pd.DataFrame.from_dict({'':comp_review_dict}, orient='index') #데이터를 데이터프레임으로 저장
    return df

## Test

In [ ]:
df = get_url("(주)레몬")
df